In [1]:
import os
import requests
from bs4 import BeautifulSoup
from typing import List
from dotenv import load_dotenv
from openai import OpenAI
import google.generativeai
import anthropic
from google import generativeai

In [2]:
import gradio as gr

In [3]:
load_dotenv(override=True)
openai_api_key = os.getenv('OPENAI_API_KEY')
google_api_key = os.getenv('GOOGLE_API_KEY')
if openai_api_key:
    print(f"OpenAI API Key exists and begins {openai_api_key[:8]}")
else:
    print("OpenAI API Key not set")
if google_api_key:
    print(f"Google API Key exists and begins {google_api_key[:8]}")
else:
    print("Google API Key not set")

OpenAI API Key exists and begins sk-proj-
Google API Key exists and begins AIzaSyDK


In [9]:
openai = OpenAI()
google.generativeai.configure()
system_message = "You are a helpful assistant"

In [5]:
def streamgpt(prompt):
    messages = [
        {"role": "system", "content": system_message},
        {"role": "user", "content": prompt}
      ]
    stream = openai.chat.completions.create(
        model='gpt-4o-mini',
        messages=messages,
        stream=True
    )
    result = ""
    for chunk in stream:
        result += chunk.choices[0].delta.content or ""
        yield result

In [10]:
def streamgemini(prompt):
    gemini = google.generativeai.GenerativeModel(
        model_name='gemini-1.5-flash',
        system_instruction=system_message)
    responses = gemini.generate_content(prompt, stream=True)
    result = ""
    for chunk in responses:
        result += chunk.text or ""
        yield result   

In [11]:
def stream_model(prompt, model):
    if model=="GPT":
        result = streamgpt(prompt)
    elif model=="Gemini":
        result = streamgemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [12]:
view = gr.Interface(
    fn=stream_model,
    inputs=[gr.Textbox(label="Your message:"), gr.Dropdown(["GPT", "Gemini"], label="Select model", value="GPT")],
    outputs=[gr.Markdown(label="Response:")],
    flagging_mode="never"
)
view.launch()

* Running on local URL:  http://127.0.0.1:7890
* To create a public link, set `share=True` in `launch()`.


In [13]:
class Website:
    url: str
    title: str
    text: str

    def __init__(self, url):
        self.url = url
        response = requests.get(url)
        self.body = response.content
        soup = BeautifulSoup(self.body, 'html.parser')
        self.title = soup.title.string if soup.title else "No title found"
        for irrelevant in soup.body(["script", "style", "img", "input"]):
            irrelevant.decompose()
        self.text = soup.body.get_text(separator="\n", strip=True)

    def get_contents(self):
        return f"Webpage Title:\n{self.title}\nWebpage Contents:\n{self.text}\n\n"

In [18]:
system_message = "You are an assistant that analyzes the contents of a company website landing page \
and creates a short brochure about the company for prospective customers, investors and recruits. Respond in markdown."

In [23]:
def stream_brochure(company_name, url,genre, model):
    if genre == "Funny":
        prompt = f"Please generate a company brochure for {company_name} in funny way. Here is their landing page:\n"
    if genre == "Aggressive":
        prompt = f"Please generate a company brochure for {company_name} in aggressive way. Here is their landing page:\n"
    if genre == "Sarcastic":
        prompt = f"Please generate a company brochure for {company_name} in sarcastic way. Here is their landing page:\n"
    if genre == "Sad":
        prompt = f"Please generate a company brochure for {company_name} in sad way. Here is their landing page:\n"
    
    prompt += Website(url).get_contents()
    if model=="GPT":
        result = streamgpt(prompt)
    elif model=="Gemini":
        result = streamgemini(prompt)
    else:
        raise ValueError("Unknown model")
    yield from result

In [26]:
view = gr.Interface(
    fn=stream_brochure,
    inputs=[
        gr.Textbox(label="Company name:"),
        gr.Textbox(label="Landing page URL including http:// or https://"),
        gr.Dropdown(["Funny","Aggressive","Sarcastic","Sad"],label="Select Genre"),
        gr.Dropdown(["GPT", "Gemini"], label="Select model")],
    outputs=[gr.Markdown(label="Brochure:")],
    flagging_mode="never"
)
view.launch(share=True)

* Running on local URL:  http://127.0.0.1:7894
* Running on public URL: https://9c9355b616c5fadd03.gradio.live

This share link expires in 1 week. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
